# VQA BGE-small search

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
import ast
import pickle
import re
import torch.functional as F

In [2]:
df = pd.read_csv('prepared_test_vqa_dataset.csv')
df

question  \
0                  What is the man by the bags awaiting?   
1                         Where does this man eat pizza?   
2          What is the occupation of the person driving?   
3      How were the drivers of the cars able to park ...   
4      How many people can ride this motorcycle at a ...   
...                                                  ...   
17020         The Horse and rider here are part of what?   
17021              Who need to obey the stop sign shown?   
17022       What is the red object sitting on the stove?   
17023  To which elevation will the persons pictured h...   
17024         Which player has the higher jersey number?   

                                                 choices dataset  \
0           ['train', 'skateboarder', 'delivery', 'cab']  aokvqa   
1                 ['office', 'motel', 'cafe', 'outside']  aokvqa   
2            ['musician', 'cashier', 'farmer', 'waiter']  aokvqa   
3      ['police', 'firemen', 'airport workers', 'post...  aokvqa   
4                        ['three', 'four', 'one', 'two']  aokvqa   
...                                                  ...     ...   
17020  ['rodeo roundup', 'parade', 'runaway horse', '...  aokvqa   
17021        ['cars', 'runners', 'motorcycles', 'buses']  aokvqa   
17022              ['mug', 'container', 'bag', 'teapot']  aokvqa   
17023           ['lower', 'higher', 'same', 'sea level']  aokvqa   
17024  ['baserunner', 'goalie', 'quarterback', 'pitch...  aokvqa   

                answer                                         image_path  \
0                  cab  /home/jovyan/llama_vision/val2014/COCO_val2014...   
1               office  /home/jovyan/llama_vision/train2014/COCO_train...   
2               farmer  /home/jovyan/llama_vision/train2014/COCO_train...   
3      airport workers  /home/jovyan/llama_vision/train2014/COCO_train...   
4                  two  /home/jovyan/llama_vision/val2014/COCO_val2014...   
...                ...                                                ...   
17020           parade  /home/jovyan/llama_vision/train2014/COCO_train...   
17021          runners  /home/jovyan/llama_vision/val2014/COCO_val2014...   
17022           teapot  /home/jovyan/llama_vision/train2014/COCO_train...   
17023            lower  /home/jovyan/llama_vision/train2014/COCO_train...   
17024          pitcher  /home/jovyan/llama_vision/val2014/COCO_val2014...   

       answer_index  
0                 3  
1                 0  
2                 2  
3                 2  
4                 3  
...             ...  
17020             1  
17021             1  
17022             3  
17023             0  
17024             3  

[17025 rows x 6 columns]

In [3]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(row):
    return row['question']

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_csv(path)
    df = pd.DataFrame()
    
    df['question'] = data['question']
    df['choices'] = data['choices'].apply(ast.literal_eval)
    
    return df

In [11]:
# Загружаем данные
train_data = make_df('prepared_test_vqa_dataset.csv')

# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
train_choices = [prepare_text(x) for x in np.concatenate(train_data['choices'])]

In [12]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    """Функция, для получения векторных представлений из текста"""
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [13]:
encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')

2024-11-09 19:29:02.956397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 19:29:02.956451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 19:29:02.957598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 19:29:02.963684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-09 19:29:03.951777: W tensorflow/compiler/tf2

In [14]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [15]:
# делаем препроцессинг вершин
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]

In [16]:
features = create_embeds(concepts,encoder,tokenizer,batch_size=512)

  0%|          | 0/1562 [00:00<?, ?it/s]

In [17]:
# Создаём faiss index для быстрого поиска кандидатов
index = faiss.IndexFlatIP(features.shape[1])
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [18]:
# Векторизуем запросы
question_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
choice_features = create_embeds(train_choices,encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/533 [00:00<?, ?it/s]

In [20]:
# Ищем по запросам релевантные вершины 
faiss.normalize_L2(question_features)
faiss.normalize_L2(choice_features)
D_q, I_q = index.search(question_features,k=7)
D_ch, I_ch = index.search(choice_features,k=1)

In [23]:
I_ch[:,0]

array([17241, 28258,  6586, ..., 22493, 48514, 47639])

In [25]:
state_bge = {
    'question_encodes': I_q,
    'choices_encodes': I_ch[:,0]
}

with open('./vqa_state_bge','wb') as f:
    pickle.dump(state_bge,f)

Если у вас мало RAM, то рекомендуется перезагрузить среду

# Lemmas Search

In [8]:
import numpy as np
import pandas as pd
import re
import multiprocessing as mp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import spacy
import pickle
import ast
from string import punctuation
from spacy.lang.en import stop_words

In [9]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(row):
    return row['question']

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_csv(path)
    df = pd.DataFrame()
    
    df['question'] = data['question']
    df['choices'] = data['choices'].apply(ast.literal_eval)
    
    return df

In [10]:
# Загружаем данные
train_data = make_df('prepared_test_vqa_dataset.csv')

# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)

In [11]:
class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        """Базовая предобработка текста перед лемматизацией"""
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        """Лемматизируем запрос и удаляем стоп слова"""
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

In [12]:
class EntityLinlker():
    def __init__(self, kg, text_proc, prepared2concept):
        self.kg = kg
        self.text_proc = text_proc
        self.p2c = prepared2concept
        
    def link_sentence(self,s):
        """Ищем для запроса релевантных кандидатов"""
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts

In [13]:
processor = TextProcessor()
graph_state = pickle.load(open('graph.pkl','rb'))
concepts = graph_state['concepts_maper'].keys()
concepts = [(x if type(x) == str else '') for x in concepts] #delete valid nodes

In [14]:
prepared_concepts = pickle.load(open('entity_linker_kg.pkl','rb')) # - если не хотите ждать
concept2prepared = dict(zip(concepts,prepared_concepts)) # словарь, сопоставляющий концепт -> его лемматезированная версия

In [15]:
def texual_acc(s1,s2): # like charackter bleu@1 score
    return sum(i == j for i,j in zip(s1,s2)) / (max(len(s1),len(s2)) + 1e-6)

def prep_text(s):
    return s.lower().strip().replace('_', ' ')

def rank_func(s1,texts):
    s1 = prep_text(s1)
    scores = []
    for s in texts:
        scores.append(texual_acc(s1,prep_text(s)))
    return texts[np.argmax(scores)]

In [16]:
# из за лемматизации, текстовая составляющая некоторых вершин стала идентичной
# по этому для их текста ищем вершину, чья не лемматезированная версия как можно сильнее совподает с не лемматезированной
# таким образом получаем словарь, который сопоставляет лемматизированному концепту его исходную версию
df_cp = pd.DataFrame(zip(concepts,prepared_concepts))
df_cp[1] = df_cp[1].apply(lambda x: ' '.join(x))
df_cp_grp = df_cp.groupby(1)[0].agg(list)
ranked_nodes = [rank_func(y,x) for x,y in tzip(df_cp_grp.tolist(), df_cp_grp.index.tolist())]
prepared2concept = dict(zip(df_cp_grp.index.tolist(),ranked_nodes))

  0%|          | 0/656261 [00:00<?, ?it/s]

In [17]:
linker = EntityLinlker(prepared_concepts, processor, prepared2concept)

In [20]:
# ищем кандидатов
train_data['candidates'] = linker.batch_linking(train_data['request'].tolist())

  0%|          | 0/17025 [00:00<?, ?it/s]

In [21]:
train_data['candidates'] 

0        [931, 10597, 10598, 10622, 10802, 11352, 17213...
1        [931, 5729, 5730, 5936, 5937, 10597, 10598, 10...
2        [934, 4369, 5741, 11895, 15379, 24125, 31448, ...
3        [59, 64, 2013, 2407, 5742, 5743, 5749, 16751, ...
4        [749, 751, 754, 1510, 1513, 1515, 2098, 2274, ...
                               ...                        
17020    [4287, 23192, 23224, 27402, 28164, 34900, 3914...
17021    [561, 3619, 7206, 7208, 11276, 11277, 11388, 1...
17022    [797, 1372, 1373, 1374, 1375, 4501, 8974, 1119...
17023    [934, 4351, 4369, 5068, 5070, 6074, 10205, 121...
17024    [581, 4695, 6398, 6404, 8334, 21869, 23773, 26...
Name: candidates, Length: 17025, dtype: object

In [23]:
def get_concepts(ids):
    """Преобразуем концепты в число"""
    ids = [prepared_concepts[i] for i in ids]
    ids = [' '.join(x) for x in ids]
    return [graph_state['concepts_maper'][prepared2concept[i]] for i in ids] #[concepts.index(prepared2concept[i]) for i in ids] 

In [25]:
train_data['candidates'] = train_data['candidates'].apply(get_concepts)

In [27]:
train_data['candidates']

0        [931, 931, 931, 931, 931, 931, 17213, 931, 931...
1        [931, 5729, 5729, 5729, 5729, 931, 931, 931, 9...
2        [934, 934, 5741, 11895, 5741, 934, 934, 934, 5...
3        [59, 59, 59, 2407, 5742, 5742, 5749, 2407, 240...
4        [751, 751, 751, 2274, 2274, 2274, 2274, 2274, ...
                               ...                        
17020    [4287, 4287, 4287, 48965, 4287, 4287, 4287, 48...
17021    [561, 3619, 561, 561, 11276, 11276, 11276, 282...
17022    [797, 1372, 1372, 1372, 1372, 1372, 797, 1372,...
17023    [934, 4351, 934, 4351, 4351, 6074, 10205, 4351...
17024    [581, 4695, 581, 581, 4695, 26711, 23773, 2671...
Name: candidates, Length: 17025, dtype: object

In [32]:
idf_matrix = pd.Series(
    np.concatenate(train_data['candidates'])
).value_counts()
idf_matrix = idf_matrix.to_dict()

In [33]:
# Берём семплы с наименьшей популярностью, так как они будут более значимыми для нашей модели
# Таким образом получается получить top-k самых редких семплов, о которых исходная LM скорее всего не обладает знаниями
def rank_by_idf(texts,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [idf_matrix[x] for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

In [36]:
train_data['candidates_rank'] = train_data['candidates'].apply(rank_by_idf)

# Merge All candidates

In [41]:
bge_state = pickle.load(open('vqa_state_bge','rb'))

In [48]:
bge_state['choices_encodes']

array([17241, 28258,  6586, ..., 22493, 48514, 47639])

In [43]:
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]
concepts_inv = dict(enumerate(concepts))

In [66]:
question_bge = [[concepts_inv[x] for x in i] for i in bge_state['question_encodes']]
choices_bge = [[concepts_inv[x] for x in bge_state['choices_encodes'][i:i+4]] for i in range(0,len(bge_state['choices_encodes']),4)]

In [71]:
choices_bge[3]

['police', 'firemen', 'airport worker', 'postal workers']

In [75]:
lemma_candides = [[graph_state['concepts_maper_inv'][x] for x in k] for k in train_data['candidates_rank'].tolist()]

In [81]:
def merge_cands(x1,x2,x3):
    all_cands = np.unique(x1 + x2 + x3)
    return all_cands.tolist()

In [82]:
all_cands = [
    merge_cands(x1,x2,x3)
    for x1,x2,x3 in zip(question_bge, lemma_candides, choices_bge)
]

In [83]:
with open('vqa_candidates.pkl;all_cands

[['await',
  'bag',
  'bag man',
  'bag snatcher',
  'baggageman',
  'cab',
  'delivered in bags',
  'delivery',
  'give someone bag',
  'man',
  'man bag',
  'man_bag',
  'skateboarder',
  'train',
  'waiting for luggage'],
 ['café',
  'eat',
  'eat pizza',
  'eat_pizza',
  'eating pizza',
  'man',
  'man_eating',
  'motel',
  'office',
  'on pizza',
  'outside',
  'pizza',
  'pizza delivery man',
  'pizza guy',
  'pizza man'],
 ['ability to drive',
  'car operator',
  'cashier',
  'drive',
  'driven by drivers',
  'driving',
  'driving examiner',
  'farmer',
  'motorist',
  'musician',
  'occupation',
  'person',
  'waiter'],
 ['able',
  'airport worker',
  'car',
  'driven by drivers',
  'driver',
  'firemen',
  'park',
  'parked cars',
  'parked in car parks',
  'parking cars',
  'parking cars in',
  'police',
  'postal workers',
  'where people drive cars',
  'where people park cars'],
 ['four',
  'large motorcycle',
  'lot of rides',
  'motorcycle',
  'motorcycle ride',
  'on mot